Installing required libraries

In [1]:
!pip install -q gradio tensorflow tensorflow-addons facenet-pytorch moviepy opencv-python torch==2.3.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 56.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 59.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 44.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 9.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.1/318.1 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 50.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 66.2 MB/s

Importing the libraries

In [2]:
import gradio as gr
import cv2
import numpy as np
import tensorflow as tf
import tensorflow_addons

from facenet_pytorch import MTCNN
from PIL import Image
import moviepy.editor as mp
import os

import warnings
warnings.filterwarnings("ignore")

Mount G-Drive and Extract the pre-trained model

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!unzip /content/drive/MyDrive/DeepFake_Video_Detection/FINAL-EFFICIENTNETV2-B0.zip -d /content/saved_model

Archive:  /content/drive/MyDrive/Resume_Projects/DeepFake_Video_Detection/FINAL-EFFICIENTNETV2-B0.zip
   creating: /content/saved_model/assets/
   creating: /content/saved_model/variables/
  inflating: /content/saved_model/saved_model.pb  
  inflating: /content/saved_model/keras_metadata.pb  
  inflating: /content/saved_model/variables/variables.data-00000-of-00001  
  inflating: /content/saved_model/variables/variables.index  


Loading Face Detector model

In [5]:
mtcnn = MTCNN(margin=14, keep_all=True, factor=0.7, device='cpu')

Creating a detection pipeline

In [6]:
class DetectionPipeline:
    """Pipeline class for detecting faces in the frames of a video file."""

    def __init__(self, detector, n_frames=None, batch_size=60, resize=None):
        self.detector = detector
        self.n_frames = n_frames
        self.batch_size = batch_size
        self.resize = resize

    def __call__(self, filename):
        v_cap = cv2.VideoCapture(filename)
        v_len = int(v_cap.get(cv2.CAP_PROP_FRAME_COUNT))

        if self.n_frames is None:
            sample = np.arange(0, v_len)
        else:
            sample = np.linspace(0, v_len - 1, self.n_frames).astype(int)

        faces = []
        frames = []
        face2 = np.zeros((224, 224, 3), dtype=np.uint8)

        for j in range(v_len):
            success = v_cap.grab()
            if j in sample:
                success, frame = v_cap.retrieve()
                if not success:
                    continue
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

                if self.resize is not None:
                    frame = cv2.resize(frame, (int(frame.shape[1] * self.resize), int(frame.shape[0] * self.resize)))
                frames.append(frame)

                if len(frames) % self.batch_size == 0 or j == sample[-1]:
                    boxes, probs = self.detector.detect(frames)

                    for i in range(len(frames)):
                        if boxes[i] is None:
                            faces.append(face2)
                            continue

                        box = boxes[i][0].astype(int)
                        frame = frames[i]
                        face = frame[box[1]:box[3], box[0]:box[2]]

                        if not face.any():
                            faces.append(face2)
                            continue

                        face2 = cv2.resize(face, (224, 224))
                        faces.append(face2)

                    frames = []

        v_cap.release()
        return faces

In [7]:
detection_pipeline = DetectionPipeline(detector=mtcnn,n_frames=None, batch_size=60)

Loading saved model

In [8]:
model = tf.keras.models.load_model(r"/content/saved_model")

In [9]:
def deepfakespredict(input_video):
    faces = detection_pipeline(input_video)

    total = 0
    real = 0
    fake = 0

    for face in faces:
        face2 = face / 255.0
        pred = model.predict(np.expand_dims(face2, axis=0))[0]
        total += 1

        pred2 = pred[1]

        if pred2 > 0.5:
            fake += 1
        else:
            real += 1

    fake_ratio = fake / total

    text = ""
    text2 = "Deepfakes Confidence: " + str(fake_ratio * 100) + "%"

    if fake_ratio >= 0.5:
        text = "The video is FAKE."
    else:
        text = "The video is REAL."

    face_frames = []

    for face in faces:
        face_frame = Image.fromarray(face.astype('uint8'), 'RGB')
        face_frames.append(face_frame)

    face_frames[0].save('results.gif', save_all=True, append_images=face_frames[1:], duration=250, loop=100)
    clip = mp.VideoFileClip("results.gif")
    clip.write_videofile("video.mp4")

    return text, text2, "video.mp4"

Building Gradio Interface

In [10]:
title = "DeepConnect"
description = '''Verify the authenticity of your video using our advanced deepfake detection technology.'''

In [12]:
gr.Interface(
    fn=deepfakespredict,
    inputs=gr.Video(),
    outputs=[gr.Textbox(label="Detection Result"), gr.Textbox(label="Confidence Level"), gr.Video(label="Detected Face Sequence")],
    title=title,
    description=description
).launch(debug=False)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://bdc11be29f2be1f523.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
